In [158]:
import pandas as pd
import sklearn
from sklearn import preprocessing

In [159]:
train_data = pd.read_csv("data/train.csv",index_col=0)
train_data = train_data.fillna(0)

In [163]:
for i in train_data["last_price"].values:
    if i==0:
        print(i)

In [137]:
features=train_data.iloc[:,:-1] 
labels = train_data.iloc[:,-1:]
features = features[["last_price", "d_open_interest", "bid1","ask1", "bid1vol", "ask1vol"]]
features["bid"]=features["bid1"]<=features["last_price"]
features["ask"]=features["ask1"]>=features["last_price"]
features = features[["d_open_interest", "bid1vol", "ask1vol", "bid", "ask"]]

In [150]:

'''
X = features.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
'''
X = preprocessing.scale(features.values)
X = features.values
Y = labels.values
Y = Y.reshape(len(Y))
x_train = X[:500000]
x_val = X[500000:]
y_train = Y[:500000]
y_val = Y[500000:]


In [132]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
lr = LogisticRegression(max_iter=100000).fit(x_train, y_train)
svc = LinearSVC(max_iter=100000)
ada = AdaBoostClassifier()
mlp = MLPClassifier(alpha=1, max_iter=1000)
lr.fit(x_train, y_train)
from sklearn.metrics import f1_score
f1_score(y_val, lr.predict(x_val))

0.15981027974058656

In [141]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(x_train, y_train)
model.score(x_val,y_val)

0.6289889586490582

In [142]:
f1_score(y_val, model.predict(x_val))

0.23048944768747195

In [156]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, model.predict_proba(x_val)[:,1])

0.6201340771761394

In [157]:
sum(model.predict(x_val))

9162